# Benchmark of various outlier detection model thresholders

### The models are evaluated by ROC, Precision @ n and execution time on 17 benchmark datasets. All datasets are split (60% for training and 40% for testing). 

The thresholders covered in this example include:

1. **IQR: Inter-Quartile Region** 
2. **MAD: Median Absolute Deviation**
3. **FWFM: Full Width at Full Minimum**
4. **YJ: Yeo-Johnson Transformation**
5. **ZSCORE: Z Score**
6. **AUCP: Area Under the Curve Percentage**
7. **QMCD: Quasi-Monte Carlo Discreperancy**
8. **FGD: Fixed Gradient Descent**
9. **DSN: Distance Shift from Normal**
10. **CLF: Trained Classifier**
11. **FILTER: Filtering Based**
12. **WIND: Topological Winding Number**
13. **EB: Elliptical Boundary**
14. **REGR: Regression Intercept**
15. **BOOT: Bootstrap Method**
16. **MCST: Monte Carlo Statistical Tests**
17. **HIST: Histogram Based Methods**
18. **MOLL: Mollifier**
19. **CHAU: Chauvenet's Criterion**
20. **GESD: Generalized Extreme Studentized Deviate**
21. **MTT: Modified Thompson Tau Test**
22. **KARCH: Karcher Mean**
23. **OCSVM: One-Class SVM**
24. **CLUST: Clustering**
25. **DECOMP: Decomposition**
26. **META: Meta-model**

In [2]:
from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# temporary solution for relative imports in case pyod is not installed
# if pythresh is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))

import numpy as np
from numpy import percentile
import matplotlib.pyplot as plt
import matplotlib.font_manager
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.knn import KNN
from pyod.models.iforest import IForest

rom pythresh.thresholds.iqr import IQR
from pythresh.thresholds.mad import MAD
from pythresh.thresholds.fwfm import FWFM
from pythresh.thresholds.yj import YJ
from pythresh.thresholds.zscore import ZSCORE
from pythresh.thresholds.aucp import AUCP
from pythresh.thresholds.qmcd import QMCD
from pythresh.thresholds.fgd import FGD
from pythresh.thresholds.dsn import DSN
from pythresh.thresholds.clf import CLF
from pythresh.thresholds.filter import FILTER
from pythresh.thresholds.wind import WIND
from pythresh.thresholds.eb import EB
from pythresh.thresholds.regr import REGR
from pythresh.thresholds.boot import BOOT
from pythresh.thresholds.mcst import MCST
from pythresh.thresholds.hist import HIST
from pythresh.thresholds.moll import MOLL
from pythresh.thresholds.chau import CHAU
from pythresh.thresholds.gesd import GESD
from pythresh.thresholds.mtt import MTT
from pythresh.thresholds.karch import KARCH
from pythresh.thresholds.ocsvm import OCSVM
from pythresh.thresholds.clust import CLUST
from pythresh.thresholds.decomp import DECOMP
from pythresh.thresholds.meta import META

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [3]:
# Define data file and read X and y
mat_file_list = ['arrhythmia.mat',
                 'cardio.mat',
                 'glass.mat',
                 'ionosphere.mat',
                 'letter.mat',
                 'lympho.mat',
                 'mnist.mat',
                 'musk.mat',
                 'optdigits.mat',
                 'pendigits.mat',
                 'pima.mat',
                 'satellite.mat',
                 'satimage-2.mat',
                 'vertebral.mat',
                 'vowels.mat',
                 'wbc.mat']

# Define nine outlier detection tools to be compared
random_state = np.random.RandomState(42)

df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc', 'IQR', 'MAD', 'FWFM', 
              'YJ', 'ZSCORE', 'AUCP', 'QMCD', 'FGD', 'DSN', 'CLF', 'FILTER', 'WIND', 
              'EB', 'REGR', 'BOOT', 'MCST', 'HIST', 'MOLL', 'CHAU', 'GESD', 'MTT', 
              'KARCH', 'OCSVM', 'CLUST', 'DECOMP', 'META']

roc_df = pd.DataFrame(columns=df_columns)
prn_df = pd.DataFrame(columns=df_columns)
time_df = pd.DataFrame(columns=df_columns)

clf = IForest()


for mat_file in mat_file_list:
    print("\n... Processing", mat_file, '...')
    mat = loadmat(os.path.join('data', mat_file))

    X = mat['X']
    y = mat['y'].ravel()
    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)

    # construct containers for saving results
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

    # 60% data for training and 40% for testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,
                                                        random_state=random_state)

    # standardizing data for processing
    X_train_norm, X_test_norm = standardizer(X_train, X_test)

    thresholders = {
        'Inter-Quartile Region (IQR)':IQR(),
        'Median Absolute Deviation (MAD)':MAD(),
        'Full Width at Full Minimum (FWFM)':FWFM(),
        'Yeo-Johnson Transformation (YJ)': YJ(),
        'Z Score (ZSCORE)': ZSCORE(),
        'AUC Percentage (AUCP)': AUCP(),
        'Quasi-Monte Carlo Discreperancy (QMCD)': QMCD(),
        'Fixed Gradient Descent (FGD)': FGD(),
        'Distance Shift from Normal (DSN)': DSN(),
        'Trained Classifier (CLF)': CLF(),
        'Filtering Based (FILTER)': FILTER(),
        'Topological Winding Number (WIND)': WIND(),
        'Elliptical Boundary (EB)': EB(),
        'Regression Intercept (REGR)': REGR(),
        'Bootstrap Method (BOOT)': BOOT(),
        'Monte Carlo Statistical Tests (MCST)': MCST(),
        'Histogram Based Methods (HIST)': HIST(),
        'Mollifier (MOLL)': MOLL(),
        "Chauvenet's Criterion (CHAU)": CHAU(),
        'Generalized Extreme Studentized Deviate (GESD)': GESD(),
        'Modified Thompson Tau Test (MTT)': MTT(),
        'Karcher Mean (KARCH)': KARCH(),
        'One-Class SVM (OCSVM)': OCSVM(),
        'Clustering (CLUST)': CLUST(),
        'Decomposition (DECOMP)': DECOMP(),
        'Meta-model (META)': META()
    }
    
    clf.fit(X_train_norm)
    scores = clf.decision_scores_
    
    for thres_name, thres in thresholders.items():
        t0 = time()
        pred = thres.eval(scores)
        contam = np.sum(pred)/len(pred)
        
        if contam<=0: contam=1e-3
        if contam>0.5: contam=0.5
        
        clf = IForest(contamination=contam)
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1 - t0, ndigits=4)
        time_list.append(duration)

        roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
        prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

        print('{thres_name} ROC:{roc}, precision @ rank n:{prn}, '
              'execution time: {duration}s'.format(
        thres_name=thres_name, roc=roc, prn=prn, duration=duration))

        roc_list.append(roc)
        prn_list.append(prn)

    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)

    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)

    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)


... Processing arrhythmia.mat ...
Inter-Quartile Region (IQR) ROC:0.8385, precision @ rank n:0.5, execution time: 0.8303s
Median Absolute Deviation (MAD) ROC:0.8249, precision @ rank n:0.5, execution time: 0.8159s
Full Width at Full Minimum (FWFM) ROC:0.8445, precision @ rank n:0.5, execution time: 0.7702s
Yeo-Johnson Transformation (YJ) ROC:0.845, precision @ rank n:0.5357, execution time: 1.1663s
Z Score (ZSCORE) ROC:0.8235, precision @ rank n:0.5357, execution time: 0.8017s
AUC Percentage (AUCP) ROC:0.8389, precision @ rank n:0.5357, execution time: 0.8607s
Quasi-Monte Carlo Discreperancy (QMCD) ROC:0.8487, precision @ rank n:0.5714, execution time: 0.806s
Fixed Gradient Descent (FGD) ROC:0.8126, precision @ rank n:0.5357, execution time: 0.7758s
Distance Shift from Normal (DSN) ROC:0.8506, precision @ rank n:0.5357, execution time: 0.7836s
Trained Classifier (CLF) ROC:0.8455, precision @ rank n:0.5714, execution time: 0.8313s
Filtering Based (FILTER) ROC:0.8415, precision @ rank n

Elliptical Boundary (EB) ROC:0.7931, precision @ rank n:0.587, execution time: 0.9665s
Regression Intercept (REGR) ROC:0.7924, precision @ rank n:0.5652, execution time: 0.6125s
Bootstrap Method (BOOT) ROC:0.7888, precision @ rank n:0.5435, execution time: 0.6457s
Monte Carlo Statistical Tests (MCST) ROC:0.7803, precision @ rank n:0.5217, execution time: 0.5868s
Histogram Based Methods (HIST) ROC:0.7684, precision @ rank n:0.5435, execution time: 0.5665s
Mollifier (MOLL) ROC:0.7757, precision @ rank n:0.5217, execution time: 0.5832s
Chauvenet's Criterion (CHAU) ROC:0.768, precision @ rank n:0.5217, execution time: 0.6468s
Generalized Extreme Studentized Deviate (GESD) ROC:0.7817, precision @ rank n:0.587, execution time: 0.6853s
Modified Thompson Tau Test (MTT) ROC:0.768, precision @ rank n:0.5, execution time: 0.6005s
Karcher Mean (KARCH) ROC:0.7799, precision @ rank n:0.5217, execution time: 0.6393s
One-Class SVM (OCSVM) ROC:0.7757, precision @ rank n:0.5217, execution time: 0.8558s


Meta-model (META) ROC:0.8005, precision @ rank n:0.2815, execution time: 5.3517s

... Processing musk.mat ...
Inter-Quartile Region (IQR) ROC:1.0, precision @ rank n:1.0, execution time: 1.4904s
Median Absolute Deviation (MAD) ROC:0.9997, precision @ rank n:0.9512, execution time: 1.5708s
Full Width at Full Minimum (FWFM) ROC:1.0, precision @ rank n:1.0, execution time: 1.8002s
Yeo-Johnson Transformation (YJ) ROC:0.9992, precision @ rank n:0.925, execution time: 2.5119s
Z Score (ZSCORE) ROC:1.0, precision @ rank n:1.0, execution time: 1.4342s
AUC Percentage (AUCP) ROC:0.9999, precision @ rank n:0.9756, execution time: 1.9461s
Quasi-Monte Carlo Discreperancy (QMCD) ROC:0.9949, precision @ rank n:0.878, execution time: 1.4729s
Fixed Gradient Descent (FGD) ROC:1.0, precision @ rank n:1.0, execution time: 1.75s
Distance Shift from Normal (DSN) ROC:0.9994, precision @ rank n:0.9024, execution time: 2.0131s
Trained Classifier (CLF) ROC:1.0, precision @ rank n:1.0, execution time: 1.5271s
Fil

Topological Winding Number (WIND) ROC:0.7077, precision @ rank n:0.5752, execution time: 0.5974s
Elliptical Boundary (EB) ROC:0.6819, precision @ rank n:0.5664, execution time: 1.0112s
Regression Intercept (REGR) ROC:0.6989, precision @ rank n:0.5752, execution time: 0.6204s
Bootstrap Method (BOOT) ROC:0.7147, precision @ rank n:0.5841, execution time: 0.6824s
Monte Carlo Statistical Tests (MCST) ROC:0.7142, precision @ rank n:0.6018, execution time: 0.5782s
Histogram Based Methods (HIST) ROC:0.7082, precision @ rank n:0.5664, execution time: 0.6369s
Mollifier (MOLL) ROC:0.7194, precision @ rank n:0.6106, execution time: 0.5829s
Chauvenet's Criterion (CHAU) ROC:0.7074, precision @ rank n:0.5664, execution time: 0.596s
Generalized Extreme Studentized Deviate (GESD) ROC:0.6931, precision @ rank n:0.5575, execution time: 0.7442s
Modified Thompson Tau Test (MTT) ROC:0.6939, precision @ rank n:0.5752, execution time: 0.5641s
Karcher Mean (KARCH) ROC:0.6996, precision @ rank n:0.5664, execut

Clustering (CLUST) ROC:0.4226, precision @ rank n:0.0833, execution time: 0.5455s
Decomposition (DECOMP) ROC:0.3998, precision @ rank n:0.0833, execution time: 0.5613s
Meta-model (META) ROC:0.4276, precision @ rank n:0.0833, execution time: 2.5209s

... Processing vowels.mat ...
Inter-Quartile Region (IQR) ROC:0.8, precision @ rank n:0.2632, execution time: 0.6123s
Median Absolute Deviation (MAD) ROC:0.7898, precision @ rank n:0.2105, execution time: 0.653s
Full Width at Full Minimum (FWFM) ROC:0.7944, precision @ rank n:0.3158, execution time: 0.6876s
Yeo-Johnson Transformation (YJ) ROC:0.8465, precision @ rank n:0.2105, execution time: 1.2409s
Z Score (ZSCORE) ROC:0.7713, precision @ rank n:0.3158, execution time: 0.6278s
AUC Percentage (AUCP) ROC:0.7963, precision @ rank n:0.2632, execution time: 0.7955s
Quasi-Monte Carlo Discreperancy (QMCD) ROC:0.7484, precision @ rank n:0.2632, execution time: 0.6199s
Fixed Gradient Descent (FGD) ROC:0.7802, precision @ rank n:0.2632, execution t

In [4]:
print('Time complexity')
time_df

Time complexity


,Data,#Samples,# Dimensions,Outlier Perc,IQR,MAD,FWFM,YJ,ZSCORE,AUCP,...,HIST,MOLL,CHAU,GESD,MTT,KARCH,OCSVM,CLUST,DECOMP,META
0,arrhythmia,452,274,14.6018,0.8303,0.8159,0.7702,1.1663,0.8017,0.8607,...,0.9677,0.8789,0.8055,0.895,0.8197,0.8156,0.9962,0.8105,0.8423,2.9884
0,cardio,1831,21,9.6122,0.7627,0.7559,0.8681,1.3417,0.7812,0.9796,...,0.7542,0.7481,0.8123,1.1884,1.0775,0.9438,1.5154,1.0665,1.1095,2.9864
0,glass,214,9,4.2056,0.6136,0.5761,0.5759,0.922,0.6164,0.5719,...,0.5593,0.5672,0.5639,0.5744,0.5568,0.5307,0.706,0.5488,0.5616,2.6644
0,ionosphere,351,33,35.8974,0.6359,0.5987,0.5838,0.8265,0.576,0.6155,...,0.5665,0.5832,0.6468,0.6853,0.6005,0.6393,0.8558,0.6213,0.6506,2.7405
0,letter,1600,32,6.25,0.767,0.8125,0.8933,1.2563,0.7672,0.9643,...,0.7531,0.8101,0.7553,1.1009,0.7663,0.8368,1.299,0.8831,1.0183,3.1237
0,lympho,148,18,4.0541,0.5529,0.5618,0.5873,0.7895,0.6041,0.5811,...,0.5796,0.6394,0.5683,0.5899,0.5455,0.5713,0.7272,0.5557,0.5797,2.7444
0,mnist,7603,100,9.2069,2.2025,2.2942,4.4822,5.5735,2.3281,4.4632,...,2.2729,2.443,2.3161,3.5003,2.0962,2.991,5.3903,6.4214,5.6004,5.3517
0,musk,3062,166,3.1679,1.4904,1.5708,1.8002,2.5119,1.4342,1.9461,...,1.5574,1.4657,1.491,2.0083,1.4599,1.5942,2.3292,1.9503,2.1968,3.6488
0,optdigits,5216,64,2.8758,1.3964,1.4109,2.3989,3.8615,1.4714,2.5748,...,1.3933,1.394,1.4413,2.3782,1.4397,1.8367,2.9686,3.6454,3.1322,3.7339
0,pendigits,6870,16,2.2707,1.1408,1.1319,2.885,3.852,1.1715,2.953,...,1.131,1.1755,1.1284,2.4577,1.2699,1.886,3.8094,6.1211,4.1051,3.6178


In [5]:
print('ROC Performance')
roc_df

ROC Performance


,Data,#Samples,# Dimensions,Outlier Perc,IQR,MAD,FWFM,YJ,ZSCORE,AUCP,...,HIST,MOLL,CHAU,GESD,MTT,KARCH,OCSVM,CLUST,DECOMP,META
0,arrhythmia,452,274,14.6018,0.8385,0.8249,0.8445,0.845,0.8235,0.8389,...,0.8492,0.8375,0.8548,0.8576,0.8147,0.8403,0.8172,0.8466,0.8345,0.8597
0,cardio,1831,21,9.6122,0.9294,0.9102,0.9381,0.9161,0.9181,0.9082,...,0.9236,0.9038,0.942,0.9304,0.9261,0.9034,0.9179,0.9274,0.901,0.9221
0,glass,214,9,4.2056,0.6346,0.6765,0.6222,0.6667,0.6617,0.6889,...,0.642,0.6469,0.7037,0.6494,0.6346,0.6519,0.642,0.6296,0.6198,0.6395
0,ionosphere,351,33,35.8974,0.7842,0.7913,0.8018,0.7936,0.7796,0.787,...,0.7684,0.7757,0.768,0.7817,0.768,0.7799,0.7757,0.7888,0.7723,0.7886
0,letter,1600,32,6.25,0.643,0.6364,0.5975,0.6335,0.5952,0.6258,...,0.5988,0.5677,0.6585,0.5961,0.6206,0.594,0.6753,0.627,0.6129,0.6883
0,lympho,148,18,4.0541,0.9942,1.0,0.9942,1.0,1.0,1.0,...,1.0,1.0,0.9942,1.0,1.0,0.9942,0.9942,0.9942,1.0,1.0
0,mnist,7603,100,9.2069,0.771,0.7885,0.7896,0.8006,0.825,0.7748,...,0.8053,0.8196,0.7826,0.8127,0.7941,0.7692,0.7976,0.8062,0.7882,0.8005
0,musk,3062,166,3.1679,1.0,0.9997,1.0,0.9992,1.0,0.9999,...,0.9886,0.9996,0.9997,0.9997,0.9988,1.0,1.0,1.0,1.0,1.0
0,optdigits,5216,64,2.8758,0.7461,0.6624,0.6121,0.7007,0.6863,0.7207,...,0.7732,0.6948,0.7126,0.7197,0.7483,0.6892,0.7659,0.7221,0.7606,0.6923
0,pendigits,6870,16,2.2707,0.9595,0.9487,0.9437,0.9331,0.9667,0.9459,...,0.9497,0.9349,0.9256,0.9442,0.9437,0.959,0.9222,0.9528,0.943,0.9452


In [6]:
print('Precision @ n Performance')
prn_df

Precision @ n Performance


,Data,#Samples,# Dimensions,Outlier Perc,IQR,MAD,FWFM,YJ,ZSCORE,AUCP,...,HIST,MOLL,CHAU,GESD,MTT,KARCH,OCSVM,CLUST,DECOMP,META
0,arrhythmia,452,274,14.6018,0.5,0.5,0.5,0.5357,0.5357,0.5357,...,0.5714,0.5714,0.6071,0.5357,0.5357,0.5357,0.5,0.5714,0.5357,0.5357
0,cardio,1831,21,9.6122,0.5286,0.4143,0.5571,0.4714,0.4571,0.4857,...,0.5,0.4714,0.5857,0.5,0.5286,0.4429,0.4857,0.4857,0.4,0.5143
0,glass,214,9,4.2056,0.2,0.2,0.2,0.2,0.2,0.2,...,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2
0,ionosphere,351,33,35.8974,0.5435,0.5652,0.587,0.5652,0.5435,0.5652,...,0.5435,0.5217,0.5217,0.587,0.5,0.5217,0.5217,0.5435,0.5435,0.587
0,letter,1600,32,6.25,0.0976,0.0732,0.0732,0.122,0.0976,0.0488,...,0.0488,0.0732,0.0976,0.0488,0.0488,0.0488,0.0976,0.122,0.0976,0.0732
0,lympho,148,18,4.0541,0.6667,1.0,0.6667,1.0,1.0,1.0,...,1.0,1.0,0.6667,1.0,1.0,0.6667,0.6667,0.6667,1.0,1.0
0,mnist,7603,100,9.2069,0.2259,0.2815,0.2889,0.2778,0.3222,0.2407,...,0.3185,0.3556,0.2556,0.3185,0.2852,0.2407,0.2815,0.3111,0.2704,0.2815
0,musk,3062,166,3.1679,1.0,0.9512,1.0,0.925,1.0,0.9756,...,0.7561,0.9024,0.9756,0.9512,0.9024,1.0,1.0,1.0,1.0,0.975
0,optdigits,5216,64,2.8758,0.0154,0.0154,0.0,0.0308,0.0154,0.0308,...,0.0154,0.0154,0.0154,0.0462,0.0154,0.0154,0.0308,0.0154,0.0462,0.0154
0,pendigits,6870,16,2.2707,0.3387,0.2903,0.2903,0.2419,0.3065,0.2419,...,0.2742,0.2581,0.2097,0.2581,0.2419,0.3387,0.2419,0.2258,0.2742,0.3226
